<a href="https://www.kaggle.com/code/eneogbe/european-football-data-analysis-using-sql?scriptVersionId=142586701" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Step 1
### Connect to database and explore the data

In [ ]:
database = "/kaggle/input/tables/database.sqlite"  
 
conn = sqlite3.connect(database)


In [ ]:
#View tables in the database
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

## Step 2
### Write Queries to extract desired data

### 1. List of countries and their Leagues
Using the JOIN satement

In [ ]:
countries_and_leagues = pd.read_sql("""SELECT Country.id as country_id, country.name as country_name, League.name as league_name 
FROM Country
JOIN League ON Country.id = League.country_id;
""", conn)
countries_and_leagues

### 2. List of matches from the England Premier League
Using JOIN, WHERE, ORDER BY AND LIMIT

In [ ]:
england_premier_league_matches = pd.read_sql("""SELECT Match.id, Country.name AS country_name, League.name AS league_name, date, season, stage, 
T.team_long_name AS  home_team, TS.team_long_name AS away_team, home_team_goal, 
away_team_goal FROM Match
JOIN Country on Country.id = Match.country_id
JOIN League on League.id = Match.league_id
LEFT JOIN Team AS T on T.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS TS on TS.team_api_id = Match.away_team_api_id
WHERE country_name = 'England'
ORDER by date""", conn)

england_premier_league_matches

### 3. Number of matches played in the different leagues by season
Using COUNT, JOIN, GROUP BY. ORDER BY

In [ ]:
match_count_by_league = pd.read_sql("""SELECT Country.name AS country_name, League.name AS league_name, season,
count(DISTINCT match.id) as number_of_matches,
count(distinct stage) AS number_of_stages,
count(distinct HT.team_long_name) AS number_of_teams, 
sum(home_team_goal) AS total_home_team_goals, 
sum(away_team_goal) AS total_away_team_goals, 
avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
avg(home_team_goal+away_team_goal) AS avg_goals, 
sum(home_team_goal+away_team_goal) AS total_goals
FROM Match
JOIN Country on Country.id = Match.country_id
JOIN League on League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE country_name in ('Spain', 'Germany', 'France', 'Italy', 'England')
GROUP BY Country.name, League.name, season
HAVING count(distinct stage) > 10
ORDER BY Country.name, League.name, season DESC;""", conn)

match_count_by_league

### 4. Number of wins by each Premier League team for the 2008/2009 season
Using sub-queries

In [ ]:
wins_per_team = pd.read_sql("""SELECT country_name, league_name, date,
    season, home_team, away_team, home_team_goal, away_team_goal, winning_team,
    count(winning_team) AS wins
FROM (SELECT Country.name AS country_name, League.name AS league_name, date,
        season, T.team_long_name AS home_team, TS.team_long_name AS away_team,
        home_team_goal, away_team_goal,
        CASE 
            WHEN home_team_goal > away_team_goal THEN T.team_long_name
            WHEN home_team_goal < away_team_goal THEN TS.team_long_name
            ELSE 'Draw'
        END AS winning_team
    FROM Match
    JOIN Country ON Country.id = Match.country_id
    JOIN League ON League.id = Match.league_id
    LEFT JOIN Team AS T ON T.team_api_id = Match.home_team_api_id
    LEFT JOIN Team AS TS ON TS.team_api_id = Match.away_team_api_id
    WHERE country_name = 'England'
) AS matches_with_winners
GROUP BY
    season, winning_team
ORDER BY season, wins DESC;""", conn)

wins_per_team

# Step 3
## Data Visualization

### 1. Average goal per game over time

In [ ]:
df = pd.DataFrame(index=np.sort(match_count_by_league['season'].unique()), columns=match_count_by_league['country_name'].unique())

df.loc[:,'Germany'] = list(match_count_by_league.loc[match_count_by_league['country_name']=='Germany','avg_goals'])
df.loc[:,'Spain']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='Spain','avg_goals'])
df.loc[:,'France']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='France','avg_goals'])
df.loc[:,'Italy']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='Italy','avg_goals'])
df.loc[:,'England']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='England','avg_goals'])

df.plot(figsize=(12,5),title='Average Goals per Game Over Time')

### 2. Average goal difference, home and away

In [ ]:
df = pd.DataFrame(index=np.sort(match_count_by_league['season'].unique()), columns=match_count_by_league['country_name'].unique())

df.loc[:,'Germany'] = list(match_count_by_league.loc[match_count_by_league['country_name']=='Germany','avg_goal_dif'])
df.loc[:,'Spain']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='Spain','avg_goal_dif'])
df.loc[:,'France']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='France','avg_goal_dif'])
df.loc[:,'Italy']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='Italy','avg_goal_dif'])
df.loc[:,'England']   = list(match_count_by_league.loc[match_count_by_league['country_name']=='England','avg_goal_dif'])

df.plot(figsize=(12,5),title='Average Goals Difference Home And Away')